In [97]:
# 1.Youll have to install plot.ly for the map:    pip install plotly
# 2.Add your Twitter keys
# 3.Download the JSON file and put it in the same directory as this notebook https://gist.github.com/Miserlou/c5cd8364bf9b2420bb29   shoutout to this human for making the cities json file
# 4.Make your output directory for the CSV
# 5.Change jsonlen to the number of cities that you want to do, full list is 1000 test it with five first
# 6.Change search_term to your search term 
# 7. Run the code. should take about 6 hours. Nothing will show up in the CSV file till it's done. just let it do its thing
# 7. In next cell change the csv file to your output file for pandas
# 8. In the plotly cell just change the title of the map. there should be a new window that opens with the map. 
# 9. In the top righthand corner of the windoe there is a button to download it as a PNG. 
# 10. Profit.


import tweepy
import json
import csv
import time
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


# Twitter API Keys
consumer_key =
consumer_secret =
access_token = 
access_token_secret = 

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Open/create a file to append data to
csvFile = open('PUT YOUR OUTPUT FILE HERE', 'w', encoding='utf-8')  #make 'a' to addend   put your output file here

cities = json.load(open('YOUR DIRECTORY/cities.json'))  #Load the cities.json file here
citylist = []
latlong = []
i = 0
distance = '20mi'
jsonlen = 10              #change this number to count of cities in the full list are for the full list

while i < jsonlen:       
    city = cities[i]['city']
    citylist.append(city)
    
    lat = cities[i]['latitude']
    long = cities[i]['longitude']
    state = cities[i]['state']
    geocode = (str(lat) + "," + str(long) + "," + str(distance)) #do not fucking put spaces in this shit. it will not fucking work. 
    latlong.append(geocode)
    i += 1
    
city_dict = dict(zip(citylist, latlong))  
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['location', 'lat', 'long', 'tweet created at time', 'tweet text', 'tweet sentiment','tweet.user.geo_enabled'])

c = 0
search_term = ['RealDonaldTrump']       #put your search term here
for key, value in city_dict.items():
    testlat = value
    t = 0
    c+=1
    print(f'{key} is {c} out of {jsonlen} cities')
    for tweet in tweepy.Cursor(api.search,
                               q = search_term,
                               geocode=str(testlat),
                               #count=100,
                               lang = "en").items(100):
        
        if not tweet.retweeted and 'RT @' not in tweet.text:
        # Write a row to the CSV file. I use encode UTF-8
            csvWriter.writerow([key, testlat[0:10], testlat[11:22], tweet.created_at, tweet.text.encode('utf-8'), analyzer.polarity_scores(tweet.text)["compound"], tweet.user.geo_enabled]) #testlat is split into lat and long
            t += 1
    print(str(t) + ' results')     
    csvWriter.writerow        
csvFile.close()

print(api.rate_limit_status()['resources']['search']['/search/tweets'])  
reset_time = api.rate_limit_status()['resources']['search']['/search/tweets']['reset']
print('reset time: ' + time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(reset_time)))

New York is 1 out of 10 cities
28 results
Los Angeles is 2 out of 10 cities
19 results
Chicago is 3 out of 10 cities
44 results
Houston is 4 out of 10 cities
55 results
Philadelphia is 5 out of 10 cities
72 results
Phoenix is 6 out of 10 cities
59 results
San Antonio is 7 out of 10 cities
87 results
San Diego is 8 out of 10 cities
78 results
Dallas is 9 out of 10 cities
75 results
San Jose is 10 out of 10 cities
37 results
{'limit': 180, 'remaining': 47, 'reset': 1523819521}
reset time: 2018-04-15 12:12:01


In [98]:
import pandas as pd
import numpy as np
import plotly.plotly
df = pd.read_csv('PUT YOUR OUTPUT CSV FILE HERE.csv')     #Just put your output file here the rest should run
#print(df.head())
df.reset_index()

avgsent = df.groupby(['location'], as_index=True).agg(                             #get the mean sentiment for each city
                      {'tweet sentiment':['mean']})
avgcount = df.groupby(['location'], as_index=True).agg(                          #get a count of how many tweets there are for circle size
                      {'tweet sentiment':['count']})

lat = df.groupby(['location'], as_index=True)['location', 'lat'].head(1)         # get the lat this took like two hours fuck you pandas
lat1 = lat.set_index(['location'])

long = df.groupby(['location'], as_index=True)['location', 'long'].head(1)     #same for long
long1 = long.set_index(['location'])

formap = pd.concat([avgsent, avgcount, lat1, long1], axis=1)
latlong = pd.concat([lat1, long1],axis=1,)

formap                                  #this is the cleaned df for the map                     
formap = formap.rename( columns={"(tweet sentiment, mean)": "avg sentiment", "(tweet sentiment, count)": "count"})
formap = formap.reset_index()
formap.columns = ['city', 'mean sentiment', 'count', 'lat', 'long']
formap['mean sentiment'] = formap['mean sentiment'].apply(lambda x: x*100) #make the mean sentiment go from -100 to 100
formap.shape
formap.head()


,city,mean sentiment,count,lat,long
0,Chicago,-10.237500,44,41.8781136,-87.629798
1,Dallas,-4.300800,75,32.7766642,-96.796988
2,Houston,-0.467091,55,29.7604267,-95.369803
3,Los Angeles,-8.801579,19,34.0522342,-118.243684
4,New York,-2.568929,28,40.7127837,-74.005941


In [100]:
#Everything should be good to go here just change the title

formap['text'] = formap['city'] + '<br>Mean Sentiment: ' + (formap['mean sentiment']).astype(str) + '<br>Tweet Count: ' + (formap['count']).astype(str)
limits = [(0,10000)]
cities = []


for i in range(len(limits)):
    
    lim = limits[i]
    formap_sub = formap[lim[0]:lim[1]]  
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = formap_sub['long'],         
        lat = formap_sub['lat'],
        text=formap_sub['text'],
        mode='markers',
        marker = dict(
            size = formap_sub['count'] * 2,
            opacity = .7,
            color = formap_sub['mean sentiment'],
            colorscale='Blackbody',
            showscale=True, 
            reversescale = False,
            colorbar= dict(title = 'Sentiment Score <br>(100 = Positive) <br>(-100 = Negative)', titleseide = 'top'),
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
    )       
    cities.append(city)

layout = dict(
        title = 'Facebook Sentiment',             #change the title here. 
        showlegend = False,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
            
        ),
    )

fig = dict( data=cities, layout=layout )
plotly.offline.plot( fig, validate=False, filename='d3-bubble-map-populations.html' )

'file:///Users/justin/Documents/2018/ucsd/GroupProject1/d3-bubble-map-populations.html'